In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
from keras.models import Sequential
from keras import backend as K
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History 
history = History()
from sklearn import grid_search
from keras import backend as K
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')
import time
from datetime import datetime

seed = 42
np.random.seed(seed)

number_of_classes = 10

Using TensorFlow backend.
/Users/ram/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ram/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

In [3]:
def create_model (optimizer, dense_layer_sizes, filters, kernel_size, pool_size):
    
    now = datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    print ("optimizer : ", optimizer)
    print ("dense_layer_sizes : ", dense_layer_sizes)
    print ("filters : ", filters)
    print ("kernel_size : ", kernel_size)
    print ("pool_size : ", pool_size)
        
    model = Sequential()

    model.add(Conv2D(filters, kernel_size, input_shape=(28,28,1)))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))
  
    model.add(Conv2D(filters*2, kernel_size))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(Conv2D(filters*2, kernel_size))
    BatchNormalization(axis=-1)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    
    BatchNormalization()
    model.add(Dense(dense_layer_sizes))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.25))
    model.add(Dense(10))
    
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.summary()
    return model

In [4]:
model_clf = KerasClassifier(create_model, batch_size=32)

# Grid search 
optimizers = ['rmsprop', 'adam']
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [50, 100, 150]
#activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
dense_layer_sizes =  [32, 512]
filters = [32]
kernel_size = [3]
pool_size = [2]

param_grid = dict(optimizer=optimizers, dense_layer_sizes=dense_layer_sizes,filters=filters,kernel_size=kernel_size,pool_size=pool_size)

grid_clf  = GridSearchCV(model_clf,
                         param_grid=param_grid,
                         scoring='neg_log_loss',
                         n_jobs=1)
grid_clf.fit(X_train, Y_train)
print('The parameters of the best model are: ')
print(grid_clf.best_params_)

'''
# Evaluation using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model_clf, X_train, Y_train, cv=kfold)
print(results.mean())
'''

Current date and time : 
2018-01-01 00:00:12
('optimizer : ', 'rmsprop')
('dense_layer_sizes : ', 32)
('filters : ', 32)
('kernel_size : ', 3)
('pool_size : ', 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 3

Epoch 1/10
40000/40000 [==============================] - 211s - loss: 0.7612 - acc: 0.7550   
Epoch 2/10
40000/40000 [==============================] - 208s - loss: 0.2736 - acc: 0.9333   
Epoch 3/10
40000/40000 [==============================] - 213s - loss: 0.2014 - acc: 0.9548   
Epoch 4/10
40000/40000 [==============================] - 205s - loss: 0.1850 - acc: 0.9609   
Epoch 5/10
40000/40000 [==============================] - 130s - loss: 0.1613 - acc: 0.9635   
Epoch 6/10
40000/40000 [==============================] - 131s - loss: 0.1674 - acc: 0.9655   - ETA: 2s - loss: 
Epoch 7/10
40000/40000 [==============================] - 139s - loss: 0.1618 - acc: 0.9660   
Epoch 8/10
40000/40000 [==============================] - 140s - loss: 0.1595 - acc: 0.9675   
Epoch 9/10
40000/40000 [==============================] - 153s - loss: 0.1599 - acc: 0.9669   
Epoch 10/10
40000/40000 [==============================] - 46s    
Current date and time : 
2018-01-01 01:07:16
('optimizer : '

Epoch 1/10
40000/40000 [==============================] - 133s - loss: 0.7031 - acc: 0.7666   
Epoch 2/10
40000/40000 [==============================] - 131s - loss: 0.2399 - acc: 0.9360   
Epoch 3/10
40000/40000 [==============================] - 131s - loss: 0.1810 - acc: 0.9533   
Epoch 4/10
40000/40000 [==============================] - 131s - loss: 0.1571 - acc: 0.9596   
Epoch 5/10
40000/40000 [==============================] - 131s - loss: 0.1376 - acc: 0.9643   
Epoch 6/10
40000/40000 [==============================] - 131s - loss: 0.1277 - acc: 0.9681   
Epoch 7/10
40000/40000 [==============================] - 131s - loss: 0.1193 - acc: 0.9696   
Epoch 8/10
40000/40000 [==============================] - 131s - loss: 0.1090 - acc: 0.9727   
Epoch 9/10
40000/40000 [==============================] - 131s - loss: 0.1053 - acc: 0.9739   
Epoch 10/10
39968/40000 [============================>.] - ETA: 0sCurrent date and time : 
2018-01-01 01:53:53
('optimizer : ', 'adam')
('dense_l

Epoch 1/10
40000/40000 [==============================] - 133s - loss: 0.7470 - acc: 0.7535   
Epoch 2/10
40000/40000 [==============================] - 132s - loss: 0.2632 - acc: 0.9329   
Epoch 3/10
40000/40000 [==============================] - 130s - loss: 0.1873 - acc: 0.9547   
Epoch 4/10
40000/40000 [==============================] - 132s - loss: 0.1547 - acc: 0.9633   
Epoch 5/10
40000/40000 [==============================] - 133s - loss: 0.1440 - acc: 0.9670   
Epoch 6/10
40000/40000 [==============================] - 132s - loss: 0.1233 - acc: 0.9705   
Epoch 7/10
40000/40000 [==============================] - 133s - loss: 0.1148 - acc: 0.9733   
Epoch 8/10
40000/40000 [==============================] - 132s - loss: 0.1092 - acc: 0.9741   
Epoch 9/10
40000/40000 [==============================] - 133s - loss: 0.0937 - acc: 0.9780   
Epoch 10/10
39968/40000 [============================>.] - ETA: 0sCurrent date and time : 
2018-01-01 02:40:02
('optimizer : ', 'rmsprop')
('dens

Epoch 1/10
40000/40000 [==============================] - 156s - loss: 0.2935 - acc: 0.9142   
Epoch 2/10
40000/40000 [==============================] - 155s - loss: 0.1539 - acc: 0.9669   
Epoch 3/10
40000/40000 [==============================] - 155s - loss: 0.1650 - acc: 0.9688   
Epoch 4/10
40000/40000 [==============================] - 156s - loss: 0.1764 - acc: 0.9685   
Epoch 5/10
40000/40000 [==============================] - 155s - loss: 0.1898 - acc: 0.9658   
Epoch 6/10
40000/40000 [==============================] - 156s - loss: 0.2227 - acc: 0.9682   
Epoch 7/10
40000/40000 [==============================] - 155s - loss: 0.2160 - acc: 0.9681   
Epoch 8/10
40000/40000 [==============================] - 156s - loss: 0.2686 - acc: 0.9671   
Epoch 9/10
40000/40000 [==============================] - 156s - loss: 0.2632 - acc: 0.9654   
Epoch 10/10
39968/40000 [============================>.] - ETA: 0sCurrent date and time : 
2018-01-01 03:34:12
('optimizer : ', 'rmsprop')
('dens

Epoch 1/10
40000/40000 [==============================] - 159s - loss: 0.2440 - acc: 0.9249   
Epoch 2/10
40000/40000 [==============================] - 159s - loss: 0.0912 - acc: 0.9763   
Epoch 3/10
40000/40000 [==============================] - 159s - loss: 0.0737 - acc: 0.9811   
Epoch 4/10
40000/40000 [==============================] - 160s - loss: 0.0701 - acc: 0.9824   
Epoch 5/10
40000/40000 [==============================] - 159s - loss: 0.0613 - acc: 0.9853   
Epoch 6/10
40000/40000 [==============================] - 159s - loss: 0.0535 - acc: 0.9873   
Epoch 7/10
40000/40000 [==============================] - 159s - loss: 0.0513 - acc: 0.9869   
Epoch 8/10
40000/40000 [==============================] - 160s - loss: 0.0509 - acc: 0.9887   
Epoch 9/10
40000/40000 [==============================] - 160s - loss: 0.0450 - acc: 0.9894   
Epoch 10/10
39968/40000 [============================>.] - ETA: 0sCurrent date and time : 
2018-01-01 04:29:10
('optimizer : ', 'adam')
('dense_l

Epoch 1/10
40000/40000 [==============================] - 161s - loss: 0.2418 - acc: 0.9260   
Epoch 2/10
40000/40000 [==============================] - 160s - loss: 0.0964 - acc: 0.9739   
Epoch 3/10
40000/40000 [==============================] - 159s - loss: 0.0772 - acc: 0.9805   
Epoch 4/10
40000/40000 [==============================] - 160s - loss: 0.0656 - acc: 0.9836   
Epoch 5/10
40000/40000 [==============================] - 159s - loss: 0.0656 - acc: 0.9846   
Epoch 6/10
40000/40000 [==============================] - 160s - loss: 0.0568 - acc: 0.9861   
Epoch 7/10
40000/40000 [==============================] - 159s - loss: 0.0525 - acc: 0.9877   
Epoch 8/10
40000/40000 [==============================] - 160s - loss: 0.0471 - acc: 0.9888   
Epoch 9/10
40000/40000 [==============================] - 159s - loss: 0.0440 - acc: 0.9889   
Epoch 10/10
39968/40000 [============================>.] - ETA: 0sCurrent date and time : 
2018-01-01 05:24:45
('optimizer : ', 'adam')
('dense_l

'\n# Evaluation using 10-fold cross validation\nkfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)\nresults = cross_val_score(model_clf, X_train, Y_train, cv=kfold)\nprint(results.mean())\n'

In [5]:
# summarize results
best_clf = grid_clf.best_estimator_.model

metric_names = best_clf.metrics_names
metric_values = best_clf.evaluate(X_test, Y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)    

10000/10000 [==============================] - 10s    
('loss', ': ', 0.031316577291425868)
('acc', ': ', 0.99350000000000005)
